In [22]:
import matplotlib.pyplot as plt
import pandas as pd
import pysam
import os
import ast
import numpy as np
import json
import collections
from tqdm import tqdm
from Bio.Seq import Seq
import more_itertools as mit

In [23]:
def find_sequence(sequence, subsequence):
    positions = []
    start = 0
    while True:
        start = sequence.find(subsequence, start)
        if start == -1:
            break
        positions.append(start)
        start += 1
    return positions

In [24]:
def orientationFinder(df):
    df2 = df.copy()
    df2['Orientation']='NONE'
    for row in df2.index:
        if df2.at[row,'TE_Hits'] == 'NONE':
            continue
        else:
            elementAnnotation = str(df2.at[row,'Element_Annotation'])
            sense=0
            antisense=0
            hitList = ast.literal_eval(str(df2.at[row,'TE_Hits']))
            for hit in hitList:
                if str(hit.split()[9]) == elementAnnotation:
                    if str(hit.split()[8])=='+':
                        sense+=1
                    else:
                        antisense+=1
                else:
                    continue
            
            if sense>0 and antisense ==0:
                df2.at[row,'Orientation']='+'
            elif sense==0 and antisense>0:
                df2.at[row,'Orientation']='-'
            elif sense>antisense:
                df2.at[row,'Orientation']='+'
            elif antisense>sense:
                df2.at[row,'Orientation']='-'
            else:
                continue
    return(df2)
    

In [25]:
def tailCounter(df):
    
    df2= df.copy()
    df2['FILTER_RESULTS']='Good_Row'
    df2['Tail_Begins']='No_Tail_Detected'
    df2['Tail_Type']='No_Tail_Type'
    df2['Tail_Length']=0
    df2['Tail_Seed_Hits']=0
    
    for row in df2.index:
        

        sequence = str(df2.at[row,'Sequence']).upper()
        
        tTail = 'TTTTT'
        tTailFlag=0
        trunningTotal = 0
        tTailList=[]
        
        if tTail in sequence:
            taillength=0
            findings = [int(x) for x in find_sequence(sequence, tTail)]
            if len(findings)>0:
                tTailFlag=1
                tmin = min(findings)
                
                iterable = findings
                df2.at[row,'Tail_Seed_Hits']=len(findings)
                groupings = [list(group) for group in mit.consecutive_groups(iterable)]
                flag=0
                for group in groupings:
                    if flag==0:

                        taillength+= len(tTail) + (len(group)-1)
                        groupEnd = max(group)+len(tTail)
                        flag=1

                    else:

                        if min(group)-groupEnd<=4:
                            taillength+= (len(tTail) + (len(group)-1) + abs(min(group)-groupEnd))
                            groupEnd = max(group)+len(tTail)
                        else:
                            continue

                tTailLength = taillength
                
            else:
                pass
                
        
        aTail = 'AAAAA'
        aTailFlag=0
        arunningTotal = 0
        aTailList=[]
        if aTail in sequence:
            taillength=0
            reverseSequence = str(Seq(sequence).reverse_complement())
            findings = [int(x) for x in find_sequence(reverseSequence, tTail)]
            df2.at[row,'Tail_Seed_Hits']=len(findings)
            if len(findings)>0:
                aTailFlag=1
                amin = min(findings)
                iterable = findings
                groupings = [list(group) for group in mit.consecutive_groups(iterable)]
                flag=0
                for group in groupings:
                    if flag==0:

                        taillength+= len(aTail) + (len(group)-1)
                        groupEnd = max(group)+len(aTail)
                        flag=1

                    else:

                        if min(group)-groupEnd<=4:
                            taillength+= (len(aTail) + (len(group)-1) + abs(min(group)-groupEnd))
                            groupEnd = max(group)+len(aTail)
                        else:
                            continue

                aTailLength = taillength
                
            else:
                pass
        
        if tTailFlag>0 and aTailFlag==0:
            
            df2.at[row,'Tail_Begins']=tmin
            df2.at[row,'Tail_Type']='Possible_T-Tail'
            df2.at[row,'Tail_Length']=tTailLength
            
        elif tTailFlag==0 and aTailFlag>0:
            
            df2.at[row,'Tail_Begins']=amin
            df2.at[row,'Tail_Type']='Possible_A-Tail'
            df2.at[row,'Tail_Length']=aTailLength
        
        
        elif aTailFlag>0 and tTailFlag>0:
            
            orientation = str(df2.at[row,'Orientation'])
            
            if tTailLength<aTailLength and amin<tmin:
                df2.at[row,'Tail_Begins']=amin
                df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                df2.at[row,'Tail_Length']=aTailLength
                
            elif tTailLength>aTailLength and tmin<amin:
                df2.at[row,'Tail_Begins']=tmin
                df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                df2.at[row,'Tail_Length']=tTailLength
                
            else:
                if orientation !='None':
                    
                    if tmin>=50 and amin<50:
                        df2.at[row,'Tail_Begins']=amin
                        df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                        df2.at[row,'Tail_Length']=aTailLength
                    elif tmin<50 and amin>=50:
                        df2.at[row,'Tail_Begins']=tmin
                        df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                        df2.at[row,'Tail_Length']=tTailLength
                    
                        
                    elif aTailLength<tTailLength and orientation == '-':
                        df2.at[row,'Tail_Begins']=tmin
                        df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                        df2.at[row,'Tail_Length']=tTailLength

                    elif aTailLength>tTailLength and orientation == '+':
                        df2.at[row,'Tail_Begins']=amin
                        df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                        df2.at[row,'Tail_Length']=aTailLength
                        
                    elif orientation == '+':
                        df2.at[row,'Tail_Begins']=amin
                        df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                        df2.at[row,'Tail_Length']=aTailLength
                    
                    
                    elif orientation == '-':
                        df2.at[row,'Tail_Begins']=tmin
                        df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                        df2.at[row,'Tail_Length']=tTailLength
                    
                    else:
                        continue
                else:
                    continue
                    
                
        
        else:
            continue
            
    return(df2)
            

In [26]:
def quickFilter(df):

    df2 = df.copy()
    
    for row in df2.index:
        
        designation = str(df2.at[row, 'TE_Designation'])
        myElement = str(df2.at[row, 'Element_Annotation'])
        divergence = float(df2.at[row, 'Element_Divergence'])
        
        flags=[]
        
        #FILTER THE ALUS
        if 'SINE/Alu' == designation:

            if 'AluJ' in myElement or "FRAM" in myElement or 'FLAM' in myElement:
                flags.append("OLDER_ALU_SUBFAMILY")
            else:
                pass
            
            if int(df2.at[row,'Sequence_Length'])>500:
                flags.append("ALU_>500")
            else:
                pass

            if divergence > 6.0:
                flags.append("HIGHER_ALU_DIVERGENCE")
            else:
                pass
                
            if len(flags)>0:
                if df2.at[row,'FILTER_RESULTS']!='Good_Row':
                    tempFlagList = df2.at[row,'FILTER_RESULTS']
                    for flagg in flags:
                        tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
                else:
                    tempFlagList=[]
                    for flagg in flags:
                        tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
            else:
                continue


        #NOW FILTER THE LINES
        elif 'LINE/L1' == designation:

                
            goodLINES=['L1PA4','L1PA3','L1PA2','L1PA1', 'L1HS']
            if myElement in goodLINES:
                pass
            else:
                flags.append("OLDER_LINE_SUBFAMILY")
                
            if int(df2.at[row,'Sequence_Length'])>10000:
                flags.append("LINE_>10k")
            else:
                pass
            
            if divergence > 15.0:
                flags.append("HIGHER_LINE_DIVERGENCE")
            else:
                pass 
                
            if len(flags)>0:
                if df2.at[row,'FILTER_RESULTS']!='Good_Row':
                    tempFlagList = df2.at[row,'FILTER_RESULTS']
                    for flagg in flags:
                        tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
                else:
                    tempFlagList=[]
                    for flagg in flags:
                        tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
            else:
                continue


        #Filter SVAs
        elif 'Retroposon/SVA'==designation:

            ############################################

            if divergence > 15.0:
                flags.append("HIGHER_SVA_DIVERGENCE")
            else:
                pass 
            
            if int(df2.at[row,'Sequence_Length'])>10000:
                flags.append("SVA_>10k")
            else:
                pass
                
            if len(flags)>0:
                if df2.at[row,'FILTER_RESULTS']!='Good_Row':
                    tempFlagList = df2.at[row,'FILTER_RESULTS']
                    for flagg in flags:
                        tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
                else:
                    tempFlagList=[]
                    for flagg in flags:
                        tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
            else:
                continue


        else:

            continue

    return(df2)

In [27]:
def aluLinker(df):
    df2 = df.copy()
    
    for row in df2.index:
        
        threePrimeFlags=0
        
        if df2.at[row,'TE_Designation'] == 'SINE/Alu' and str(df2.at[row,'Tail_Type'])!='No_Tail_Type':
            
            tailDesignation = str(df2.at[row,'Tail_Type'])
            tailStartSite = int(df2.at[row,'Tail_Begins'])
            elements = ast.literal_eval(str(df2.at[row,'TE_Hits']))
            
            if len(elements)==1:
                for element in elements:

                    splitElement = element.split()

                    if abs(int(splitElement[12])-tailStartSite)>=120 and abs(int(splitElement[12])-tailStartSite)<=150:
                        threePrimeFlags+=1
                    else:
                        continue
            else:
                pass
                        
            if threePrimeFlags>0:
                df2.at[row,'FILTER_RESULTS']=['Alu_Linker_Region_Warning']
            else:
                continue
            
            
        else:
            continue
    return(df2)

In [28]:
def finalQuickCheck(df):
    
    df2 = df.copy()
    
    for row in df2.index:
            
            if str(df2.at[row,'TE_Designation']) =='LINE/L1':
                continue
                    
            elif str(df2.at[row,'TE_Designation']) =='SINE/Alu':
                
                aluList = ast.literal_eval(str(df2.at[row,'TE_Hits']))
                if len(aluList)==1:
                    continue
                else:
                    
                    aluHitList=[]
                    for hit in aluList:
                        splitHit = hit.split()
                        if 'Alu' in str(splitHit[9]):
                            aluHitList.append(splitHit[9])
                        else:
                            continue
                            
                    if len(set(aluHitList))>1:
                        if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                            df2.at[row,'FILTER_RESULTS']=['MULTI-ALU_Hits']
                        else:
                            newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                            newList.append('MULTI-ALU_Hits')
                            df2.at[row,'FILTER_RESULTS']=newList

                    else:
                        continue

    return(df2)

In [29]:
def finalQuickCheck2(df):
    
    df2 = df.copy()
    
    for row in df2.index:
            
            if str(df2.at[row,'TE_Designation']) =='LINE/L1':
                if int(df2.at[row,'Sequence_Length']) >10000:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['LINE_>10kLen']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('LINE_>10kLen')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    pass
                    
                if int(df2.at[row,'Element_Divergence']) >15.0:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['LINE_DIVERGENCE']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('LINE_DIVERGENCE')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    continue
                    
                    
            elif str(df2.at[row,'TE_Designation']) =='SINE/Alu':
                
                if int(df2.at[row,'Sequence_Length']) >500:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['ALU_>500Len']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('ALU_>500Len')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    pass
                    
                if int(df2.at[row,'Element_Divergence']) >6.0:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['ALU_DIVERGENCE']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('ALU_DIVERGENCE')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    continue
                        
                        
            elif str(df2.at[row,'TE_Designation']) =='Retroposon/SVA':
                if int(df2.at[row,'Sequence_Length']) >10000:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['SVA_>10kLen']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('SVA_>10kLen')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    pass
                    
                if int(df2.at[row,'Element_Divergence']) >15.0:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['SVA_DIVERGENCE']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('SVA_DIVERGENCE')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    continue
            
            else:
                continue

    return(df2)

In [30]:
def tailCounterCheck(df):
    df2 = df.copy()
    
    for row in df2.index:
        if df2.at[row,'Tail_Type']!='No_Tail_Type':
            flag=0

            if int(df2.at[row,'Tail_Begins'])/int(df2.at[row,'Sequence_Length'])>.5 or int(df2.at[row,'Tail_Begins'])>50:
                flag+=1
            else:
                pass

            if flag ==0:
                continue
            else:
                flagg='Bad_Tail_Position'
                if df2.at[row,'FILTER_RESULTS']!='Good_Row':
                    tempFlagList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                    tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
                else:
                    df2.at[row,'FILTER_RESULTS'] = [flagg]
        else:
            continue
    
    return(df2)

In [31]:
def cleanDataTEPercentage(df):
    
    df2 = df.copy() 
    df2['Element_Annotation']='No_Element_Annotation'
    df2['Element_Divergence']=0.0
    df2['TE_Proportion']='NONE'
    df2['TE_Percentage']=0.0
    
    #For each locus
    for row in tqdm(df2.index):
        
        if df2.at[row,'TE_Hits'] == 'NONE' or int(df2.at[row,'Sequence_Length'])>50000:
            continue
        else:
            

            columnName = 'Sequence'

            tempDict={int(x):0 for x in range(1,len(df2.at[row,columnName])+1)}    
            tempDict2={}
            tempDict3={}
            tempDivDict={}

            teHitList = ast.literal_eval(str(df2.at[row,'TE_Hits']))

            for hit in teHitList:

                splitHit = hit.split()
                focusElement = str(splitHit[9])
                tempDict3[focusElement]=str(splitHit[10])

                if focusElement in tempDict2.keys():
                    tempDivDict[focusElement].append(float(splitHit[1]))
                    pass
                else:
                    tempDivDict[focusElement]=[float(splitHit[1])]
                    tempDict2[focusElement]={x:0 for x in range(1,len(df2.at[row,columnName])+1)}
                    pass

                for coordinate in range(int(splitHit[5]), int(splitHit[6])+1):
                    tempDict[coordinate]+=1
                    tempDict2[focusElement][coordinate]+=1

            tePercentage = len([x for x in tempDict.values() if x >0])/len(tempDict)

            df2.at[row,'TE_Percentage']=float(tePercentage)

            #print(tempDict3)
            #print(tempDivDict)

            if len(tempDict2)==1:
                mykey = [x for x in tempDict2.keys()][0]
                df2.at[row,'TE_Designation']=str(tempDict3[mykey])
                df2.at[row,'TE_Proportion']={mykey:tePercentage}
                df2.at[row,'Element_Annotation']=mykey
                df2.at[row,'Element_Divergence']=np.median(tempDivDict[mykey][0])

            else:

                tempDict4 = {x:len([y for y in tempDict2[x].values() if y>0])/len(tempDict) for x in tempDict2.keys()}
                #print(tempDict4)
                maxKey = max(tempDict4, key=tempDict4.get)
                df2.at[row,'TE_Designation']=tempDict3[str(maxKey)]
                df2.at[row,'TE_Proportion']=tempDict4
                df2.at[row,'Element_Annotation']=maxKey
                df2.at[row,'Element_Divergence']=np.median(tempDivDict[maxKey])
        
    return(df2)

In [32]:
def repeatmaskerPatternFilter(df):
    df2 = df.copy()
    annotationList=[]
    df2['Unique_Element_Count']='One_Element'
    for row in df2.index:
        if df2.at[row,'TE_Proportion'] == 'NONE':
            continue
        else:

            prodict = ast.literal_eval(str(df2.at[row,'TE_Proportion']))

            if str(df2.at[row,'TE_Designation']) == 'SINE/Alu':
                aluList = [x for x in prodict.keys() if 'ALU' in str(x).upper()]
                if len(aluList)==1:
                    continue
                else:
                    df2.at[row,'Unique_Element_Count']="More_Than_One_Element"
            else:
                continue 
    
    
    for row in df2.index:
        if df2.at[row,'TE_Proportion'] == 'NONE':
            continue
        else:
        
            if df2.at[row,'Unique_Element_Count'] == 'One_Element':

                tempDict = ast.literal_eval(str(df2.at[row,'TE_Proportion']))
                dictLength = len([x for x in tempDict.keys()])

                if str(df2.at[row,'TE_Designation']) == 'SINE/Alu':
                    if len(ast.literal_eval(str(df2.at[row,'TE_Hits'])))>dictLength:
                        df2.at[row,'Unique_Element_Count'] = 'One_Element_ODD'
                    else:
                        continue



                elif str(df2.at[row,'TE_Designation']) == 'LINE/L1':
                    if len(ast.literal_eval(str(df2.at[row,'TE_Hits'])))>dictLength:
                        df2.at[row,'Unique_Element_Count'] = 'One_Element_ODD'
                    else:
                        continue

                else:
                    if len(ast.literal_eval(str(df2.at[row,'TE_Hits'])))>dictLength:
                        df2.at[row,'Unique_Element_Count'] = 'One_Element_ODD'
                    else:
                        continue


            else:
                continue
    return(df2)

In [33]:
def findTwinPriming(df):
    df2 = df.copy()
    df2['Twin_Priming_Flag']='NONE'
    
    for row in df2.index:
        if df2.at[row,'TE_Designation']=='LINE/L1':
            allhits = ast.literal_eval(str(df2.at[row,'TE_Hits']))
            orientations = []
            for hit in allhits:
                splitHit=hit.split()
                if str(splitHit[10]) == 'LINE/L1':
                    orientations.append(splitHit[8])
                else:
                    continue
            if len(set(orientations))>1:
                df2.at[row,'Twin_Priming_Flag']='FLAG'
            else:
                continue
            
            
        else:
            continue
    return(df2)

In [34]:
def simpleRepeatCheck(df):
    df2 = df.copy()
    
    for row in df2.index:
        if df2.at[row,'TE_Designation']=='Simple_repeat':
            tempDict = {x:float(y) for x,y in ast.literal_eval(str(df2.at[row,'TE_Proportion'])).items() if ')n' not in x}
            if len(tempDict)>0:
                maxKey = str(max(tempDict, key=tempDict.get)).upper()
                if 'ALU' in maxKey and len(tempDict) == 1:
                    #print(row)
                    df2.at[row,'TE_Designation']= 'SINE/Alu'
                    df2.at[row,'Element_Annotation']= maxKey

                elif'L1' in maxKey and len(tempDict) == 1:
                    #print(row)
                    df2.at[row,'TE_Designation']= 'LINE/L1'
                    df2.at[row,'Element_Annotation']= maxKey

                elif 'SVA' in maxKey and len(tempDict) == 1:
                    #print(row)
                    df2.at[row,'TE_Designation']= 'Retroposon/SVA'
                    df2.at[row,'Element_Annotation']= maxKey

                else:
                    continue
            else:
                continue
        else:
            continue
    return(df2)

In [35]:
speciesDict={
'GorGor-mat':'GorGor', 
'GorGor-pat':'GorGor', 
'PanPan-mat':'PanPan', 
'PanPan-pat':'PanPan', 
'PanTro-pri':'PanTro', 
'PanTro-alt':'PanTro', 
'PonAbe-pri':'PonAbe', 
'PonAbe-alt':'PonAbe', 
'PonPyg-alt':'PonPyg', 
'PonPyg-pri':'PonPyg', 
'SymSyn-alt':'SymSyn', 
'SymSyn-pri':'SymSyn', 
'hs1':'hs1',
}

In [54]:
directory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/'
directoryOut='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/'
for csvFile in os.listdir(directory):
    print(csvFile)
    if '50Distance_06' in str(csvFile): 
        speciesFileName = str(csvFile.split("_")[0])
        df = pd.read_csv(directory+csvFile, low_memory=False).set_index("ID")
        
        insDF_Filtered= orientationFinder(simpleRepeatCheck(cleanDataTEPercentage(df)))
        insDF_Filtered2 = tailCounter(insDF_Filtered)
        insDF_Filtered3 = aluLinker(insDF_Filtered2)
        insDF_Filtered4 = tailCounterCheck(insDF_Filtered3)
        insDF_Filtered5 = finalQuickCheck(quickFilter(insDF_Filtered4))
        insDF_Filtered6 = finalQuickCheck2(insDF_Filtered5)
        insDF_Filtered7 = repeatmaskerPatternFilter(insDF_Filtered6)
        insDF_Filtered8 = findTwinPriming(insDF_Filtered7)

        dfBad = insDF_Filtered8[insDF_Filtered8['FILTER_RESULTS']!='Good_Row'].copy()
        dfFilteredFinal = insDF_Filtered8[insDF_Filtered8['FILTER_RESULTS']=='Good_Row'].copy()
        
        tempDF = dfFilteredFinal[(dfFilteredFinal['TE_Designation']!='NONE') & (dfFilteredFinal['Tail_Type']!='No_Tail_Type')].copy()
        telist=['Retroposon/SVA','SINE/Alu','LINE/L1']
        teDF = tempDF[tempDF['TE_Designation'].isin(telist)].copy()
        teDF['UorD']='NONE'
        #break
        for row in teDF.index:
            flag=0
            if str(teDF.at[row,'All_Matches']) == 'TRUE_INSERTION':
                teDF.at[row,'UorD'] = 'Unique'
                
            else:
                
                if speciesFileName.upper() == 'HS1':
                    for species in str(teDF.at[row,'All_Matches']).split("_"):
                        if str(speciesFileName).upper() in str(species).upper() and len(ast.literal_eval(str(teDF.at[row,str(species)+'_Total_LowDistance_Hits'])))>=1:
                            flag+=1
                        else:
                            continue
                        
                    if flag ==0:
                        teDF.at[row,'UorD'] = 'Unique'
                    else:
                        teDF.at[row,'UorD'] = 'Duplication'
                    
                    
                elif speciesFileName.upper() == 'GORGOR' or speciesFileName.upper() == 'PANPAN':
                    
                    for species in str(teDF.at[row,'All_Matches']).split("_"):
                        if str(speciesFileName).upper() in str(species).upper() and len(ast.literal_eval(str(teDF.at[row,str(species)+'_Total_LowDistance_Hits'])))>1:
                            flag+=1
                        else:
                            continue
                        
                    if flag ==0:
                        teDF.at[row,'UorD'] = 'Unique'
                    else:
                        teDF.at[row,'UorD'] = 'Duplication'
                
                
                
                else:
                
                    for species in str(teDF.at[row,'All_Matches']).split("_"):
                        if str(speciesFileName).upper() in str(species).upper() and '-pri' in str(species) and len(ast.literal_eval(str(teDF.at[row,str(species)+'_Total_LowDistance_Hits'])))>=1:
                            flag+=2
                        elif str(speciesFileName).upper() in str(species).upper() and len(ast.literal_eval(str(teDF.at[row,str(species)+'_Total_LowDistance_Hits'])))>=1:
                            flag+=1
                        else:
                            continue
                        
                    if flag < 2:
                        teDF.at[row,'UorD'] = 'Unique'
                    else:
                        teDF.at[row,'UorD'] = 'Duplication'
                        
                        
                        

        currentSpecies = str(csvFile.split("_")[0])
        speciesSpecificRows=[]
        for row in teDF.index:
            flag=0
            if str(teDF.at[row,'All_Matches'])=='TRUE_INSERTION':
                speciesSpecificRows.append(row)
            else:
                for species in str(teDF.at[row,'All_Matches']).split("_"):
                    if speciesDict[species] ==currentSpecies:
                        continue
                    else:
                        flag+=1
                if flag==0:
                    speciesSpecificRows.append(row)
                else:
                    continue
                    
        finalDF = teDF.loc[speciesSpecificRows].copy()
        print(len(finalDF))
        print(collections.Counter(finalDF['All_Matches']))
        print(collections.Counter(finalDF['UorD']))
        finalDF.to_csv(directoryOut+str(currentSpecies)+"_Filtered50Distance_06_25_2024.csv")
        #dfBad.to_csv(directoryOut+str(currentSpecies)+"_Final_BadCalls_06_25_2024.csv")
        #dfFilteredFinal.to_csv(directoryOut+str(currentSpecies)+"_Fina_GoodCalls_06_25_2024.csv")
    else:
        continue

PanPan_50Distance_06-18-2024.csv


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88956/88956 [00:16<00:00, 5528.64it/s]


4092
Counter({'PanPan-pat': 1559, 'PanPan-mat': 1492, 'TRUE_INSERTION': 990, 'PanPan-mat_PanPan-pat': 51})
Counter({'Unique': 4022, 'Duplication': 70})
GorGor_50Distance_06-18-2024.csv


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101697/101697 [00:19<00:00, 5192.43it/s]


11872
Counter({'GorGor-mat': 6463, 'TRUE_INSERTION': 2725, 'GorGor-pat': 2662, 'GorGor-mat_GorGor-pat': 22})
Counter({'Unique': 11842, 'Duplication': 30})
hs1_50Distance_06-18-2024.csv


...

3743
Counter({'TRUE_INSERTION': 2605, 'PonAbe-alt': 1095, 'PonAbe-alt_PonAbe-pri': 24, 'PonAbe-pri': 19})
Counter({'Unique': 3700, 'Duplication': 43})
SymSyn_50Distance_06-18-2024.csv


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 246367/246367 [01:12<00:00, 3408.54it/s]


100512
Counter({'SymSyn-alt': 84918, 'TRUE_INSERTION': 15198, 'SymSyn-alt_SymSyn-pri': 341, 'SymSyn-pri': 55})
Counter({'Unique': 100116, 'Duplication': 396})
PonPyg_50Distance_06-18-2024.csv


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41130/41130 [00:19<00:00, 2112.62it/s]


3968
Counter({'TRUE_INSERTION': 2348, 'PonPyg-alt': 1574, 'PonPyg-pri': 33, 'PonPyg-alt_PonPyg-pri': 13})
Counter({'Unique': 3922, 'Duplication': 46})
Part3_DuplicationFiltered
old
Filtered50Distance
Part4_UniqueFiltered_Final


In [55]:
currentSpecies

'PonPyg'

In [59]:
test = finalDF[finalDF['UorD']!='Unique']

In [60]:
test[['PonPyg-alt_BestHit', 'PonPyg-alt_BestHitValue',
       'PonPyg-alt_Total_LowDistance_Hits', 'PonPyg-pri_BestHit',
       'PonPyg-pri_BestHitValue', 'PonPyg-pri_Total_LowDistance_Hits']]

,PonPyg-alt_BestHit,PonPyg-alt_BestHitValue,PonPyg-alt_Total_LowDistance_Hits,PonPyg-pri_BestHit,PonPyg-pri_BestHitValue,PonPyg-pri_Total_LowDistance_Hits
ID,,,,,,
CM054633.2:69134954-69135980,+_CM054610.2:70008431-70010464,0.6737835153922542,{},+_CM054633.2:69144667-69146693,0.02929493545183714,{'+_CM054633.2:69144667-69146693': 0.029294935...
CM054634.2:54168533-54169683,NONE,NONE,NONE,+_CM054634.2:54054548-54056699,0.014499532273152479,{'+_CM054634.2:54054548-54056699': 0.014499532...
CM054635.2:59846846-59847790,NONE,NONE,NONE,+_CM054635.2:59875260-59877208,0.011904761904761904,{'+_CM054635.2:59883839-59885788': 0.013975155...
CM054635.2:59874169-59876705,NONE,NONE,NONE,+_CM054635.2:59844752-59848287,0.009364358683314415,{'+_CM054635.2:59844752-59848287': 0.009364358...
CM054637.2:50697770-50698448,+_CM054614.2:52350479-52352158,0.017406962785114045,{'+_CM054614.2:52350479-52352158': 0.017406962...,+_CM054649.2:4415083-4416714,0.49759903961584634,{'+_CM054649.2:4415083-4416714': 0.49759903961...
CM054637.2:68348569-68349067,NONE,NONE,NONE,+_CM054637.2:68356474-68357972,0.01547779273216689,{'+_CM054637.2:68356474-68357972': 0.015477792...
CM054637.2:68357016-68357472,NONE,NONE,NONE,+_CM054637.2:68348109-68349567,0.0013850415512465374,{'+_CM054637.2:68348109-68349567': 0.001385041...
CM054640.2:35341860-35342442,+_CM054617.2:38573272-38574855,0.0006369426751592356,{'+_CM054617.2:38573272-38574855': 0.000636942...,+_CM054640.2:35351081-35352664,0.0006369426751592356,{'+_CM054640.2:35351081-35352664': 0.000636942...
CM054641.2:13459942-13465441,+_CM054623.2:20423384-20430046,0.2292276861415138,{'+_CM054623.2:20423384-20430046': 0.229227686...,+_CM054641.2:13448638-13455138,0.010944966856790504,{'+_CM054644.2:24315612-24322277': 0.234006474...


In [61]:
test[test['PonPyg-pri_BestHit']!='NONE'][['PonPyg-alt_BestHit', 'PonPyg-alt_BestHitValue',
       'PonPyg-alt_Total_LowDistance_Hits', 'PonPyg-pri_BestHit',
       'PonPyg-pri_BestHitValue', 'PonPyg-pri_Total_LowDistance_Hits']]

,PonPyg-alt_BestHit,PonPyg-alt_BestHitValue,PonPyg-alt_Total_LowDistance_Hits,PonPyg-pri_BestHit,PonPyg-pri_BestHitValue,PonPyg-pri_Total_LowDistance_Hits
ID,,,,,,
CM054633.2:69134954-69135980,+_CM054610.2:70008431-70010464,0.6737835153922542,{},+_CM054633.2:69144667-69146693,0.02929493545183714,{'+_CM054633.2:69144667-69146693': 0.029294935...
CM054634.2:54168533-54169683,NONE,NONE,NONE,+_CM054634.2:54054548-54056699,0.014499532273152479,{'+_CM054634.2:54054548-54056699': 0.014499532...
CM054635.2:59846846-59847790,NONE,NONE,NONE,+_CM054635.2:59875260-59877208,0.011904761904761904,{'+_CM054635.2:59883839-59885788': 0.013975155...
CM054635.2:59874169-59876705,NONE,NONE,NONE,+_CM054635.2:59844752-59848287,0.009364358683314415,{'+_CM054635.2:59844752-59848287': 0.009364358...
CM054637.2:50697770-50698448,+_CM054614.2:52350479-52352158,0.017406962785114045,{'+_CM054614.2:52350479-52352158': 0.017406962...,+_CM054649.2:4415083-4416714,0.49759903961584634,{'+_CM054649.2:4415083-4416714': 0.49759903961...
CM054637.2:68348569-68349067,NONE,NONE,NONE,+_CM054637.2:68356474-68357972,0.01547779273216689,{'+_CM054637.2:68356474-68357972': 0.015477792...
CM054637.2:68357016-68357472,NONE,NONE,NONE,+_CM054637.2:68348109-68349567,0.0013850415512465374,{'+_CM054637.2:68348109-68349567': 0.001385041...
CM054640.2:35341860-35342442,+_CM054617.2:38573272-38574855,0.0006369426751592356,{'+_CM054617.2:38573272-38574855': 0.000636942...,+_CM054640.2:35351081-35352664,0.0006369426751592356,{'+_CM054640.2:35351081-35352664': 0.000636942...
CM054641.2:13459942-13465441,+_CM054623.2:20423384-20430046,0.2292276861415138,{'+_CM054623.2:20423384-20430046': 0.229227686...,+_CM054641.2:13448638-13455138,0.010944966856790504,{'+_CM054644.2:24315612-24322277': 0.234006474...
